In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt

data = pd.read_excel('D:\Liberty (Base data work)\Actual data (South).xlsx')

<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:9: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Admin\AppData\Local\Temp\ipykernel_2484\860170070.py:9: SyntaxWarning: invalid escape sequence '\L'
  data = pd.read_excel('D:\Liberty (Base data work)\Actual data (South).xlsx')


In [2]:
selected_columns = ['CustomerID', 'Cleaned_Insured name', 'Policy Start Date', 'Policy End Date', 'Reg no ',
       'MANUFACTURER/Make', 'model', 'variant', 'Fuel Type', 'RTO Location ',
       'Product name ', 'Product name  2', 'biztype', 'Renewal Type', 'age',
       'Vehicle Segment', 'Number of Vehicles', 'Number of Policies',
       'Total Premium Payable (Overall)', 'Total OD Premium (Overall)',
       'Total TP Premium (Overall)', 'Total Add on Premium (Overall)', 'Average Discount (Overall)',
       'Average NCB % Previous Year (Overall)', 'Total GST (Overall)', 'Number of Claims',
       'Number of Declines', 'New Branch Name  2', 'Policy Status', 'Churn Label', 'New Customer', 'Customer Tenure ']

data = data[selected_columns]

import pandas as pd

# Convert 'Policy End Date' column to datetime format
data['Policy End Date'] = pd.to_datetime(data['Policy End Date'], errors='coerce')

# Step 1: Extract open customers for October 2024
sep_2024_mask = (
    (data['Policy End Date'].dt.year == 2024) & 
    (data['Policy End Date'].dt.month == 9)
)

open_customers = data[sep_2024_mask].copy()

# Step 2: Filter main data for 'Renewed' and 'Not Renewed' statuses up to September 2024
renewed_mask = data['Policy Status'].str.contains('Renewed|Not Renewed', case=False, na=False)
aug_2024_mask = (data['Policy End Date'] <= pd.to_datetime('2024-08-31'))

# Apply the filter and keep only the records with 'Renewed' or 'Not Renewed' statuses up to September 2024
data = data[renewed_mask & aug_2024_mask].copy()

# Convert the 'Churn Label' to binary format (1 for 'Yes', 0 for 'No')
data['Churn Label'] = data['Churn Label'].apply(lambda x: 1 if x == 'Yes' else 0)

for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna('missing')
    else:
        data[column] = data[column].fillna(0)

date_columns = ['Policy Start Date', 'Policy End Date']

for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')  

new_date_cols = {}
for col in date_columns:
    new_date_cols[f'{col}_YEAR'] = data[col].dt.year
    new_date_cols[f'{col}_MONTH'] = data[col].dt.month
    new_date_cols[f'{col}_DAY'] = data[col].dt.day

data = pd.concat([data, pd.DataFrame(new_date_cols)], axis=1)

data = data.drop(columns=date_columns)

# Define leaking features to exclude
leaking_features = ['Policy Status']

# Define features for training
features = [col for col in data.columns if col not in leaking_features + ['Churn Label']]

X = data[features]
y = data['Churn Label']

In [3]:
#label encoding for the actual data
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X[column] = label_encoder.fit_transform(X[column].astype(str))  
        label_encoders[column] = label_encoder  

C:\Users\Admin\AppData\Local\Temp\ipykernel_2484\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_2484\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_2484\2088719358.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, log_loss, confusion_matrix

import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

model = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

#Fit the model 
model.fit(X, y)

for col in date_columns:
    if col in open_customers.columns:
        open_customers[col] = pd.to_datetime(open_customers[col], errors='coerce')
    else:
        print(f"Warning: Column {col} not found in open_customers data")

open_customers_new_date_cols = {}
for col in date_columns:
    if col in open_customers.columns:
        open_customers_new_date_cols[f'{col}_YEAR'] = open_customers[col].dt.year
        open_customers_new_date_cols[f'{col}_MONTH'] = open_customers[col].dt.month
        open_customers_new_date_cols[f'{col}_DAY'] = open_customers[col].dt.day

if open_customers_new_date_cols:
    open_customers = pd.concat([open_customers, pd.DataFrame(open_customers_new_date_cols)], axis=1)

open_customers = open_customers.drop(columns=[col for col in date_columns if col in open_customers.columns])

for column in open_customers.columns:
    if open_customers[column].dtype == 'object':
        open_customers[column] = open_customers[column].fillna('missing')
    else:
        open_customers[column] = open_customers[column].fillna(0)

open_customers_encoded = open_customers.copy()

#Label Encoding for open customers
for column in open_customers_encoded.columns:
    if column in label_encoders:  
        encoder = label_encoders[column]

        mapping_dict = {label: i for i, label in enumerate(encoder.classes_)}
        next_unique_value = [max(mapping_dict.values()) + 1]  

        def encode_test_value(value):
            if value in mapping_dict:
                return mapping_dict[value]
            else:
                mapping_dict[value] = next_unique_value[0]
                next_unique_value[0] += 1
                return mapping_dict[value]
        
        open_customers_encoded[column] = open_customers_encoded[column].apply(encode_test_value)

#Predict the open customers
X_open_customers = open_customers_encoded[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

open_customers['Predicted Churn Label'] = ['Yes' if pred == 1 else 'No' for pred in y_open_pred]

print(f"Predicted No: {(y_open_pred == 0).sum()}")
print(f"Predicted Yes: {(y_open_pred == 1).sum()}")

y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

open_customers.to_csv('open_customers_predictions (South) sep.csv', index=False)

Predicted No: 195
Predicted Yes: 5550
Train Accuracy: 0.8807953189552928
Train Log Loss: 0.2637382410149754
Train ROC AUC: 0.9548462000032575
Train Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.84      0.85    113065
           1       0.90      0.91      0.90    166018

    accuracy                           0.88    279083
   macro avg       0.88      0.87      0.88    279083
weighted avg       0.88      0.88      0.88    279083

Class 0 Train Accuracy: 0.8443638614955998
Class 1 Train Accuracy: 0.9056066209688106


In [5]:
X_open_customers.columns.values

array(['CustomerID', 'Cleaned_Insured name', 'Reg no ',
       'MANUFACTURER/Make', 'model', 'variant', 'Fuel Type',
       'RTO Location ', 'Product name ', 'Product name  2', 'biztype',
       'Renewal Type', 'age', 'Vehicle Segment', 'Number of Vehicles',
       'Number of Policies', 'Total Premium Payable (Overall)',
       'Total OD Premium (Overall)', 'Total TP Premium (Overall)',
       'Total Add on Premium (Overall)', 'Average Discount (Overall)',
       'Average NCB % Previous Year (Overall)', 'Total GST (Overall)',
       'Number of Claims', 'Number of Declines', 'New Branch Name  2',
       'New Customer', 'Customer Tenure ', 'Policy Start Date_YEAR',
       'Policy Start Date_MONTH', 'Policy Start Date_DAY',
       'Policy End Date_YEAR', 'Policy End Date_MONTH',
       'Policy End Date_DAY'], dtype=object)

In [6]:
X.to_csv('X test data.csv', index=False)